In [7]:
%matplotlib inline
%xmode plain

import os
import sys
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from texter.utils import text_utils as tu
from texter.utils import embedding_utils as eu
from texter.models.keras.kim_cnn import kim_cnn

from nltk.corpus import stopwords

from keras.layers import Embedding
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from keras.callbacks import EarlyStopping

np.random.seed(15101993)

Exception reporting mode: Plain


In [8]:
excludes = stopwords.words("english")
text_path = "../../../citi/citi_rest/data/raw_data/"
text_labels = os.listdir(text_path)
pretrained_path = "../../data/others/GoogleNews-vectors-negative300-SLIM.bin"
pre_path = "../../data/others/glove.6B.50d.txt"
data = tu.load_data(excludes, text_path, text_labels)

texts = np.array([t for t in data.text])
labels = np.array([l for l in data.label])

x_train, x_test, y_train, y_test = train_test_split(texts, labels)

In [9]:
NUM_WORDS=20000
tokenizer = Tokenizer(num_words=NUM_WORDS,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'',
                      lower=True)

tokenizer.fit_on_texts(x_train)
sequences_train = tokenizer.texts_to_sequences(x_train)
sequences_valid=tokenizer.texts_to_sequences(x_test)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 57 unique tokens.


In [10]:
X_train = pad_sequences(sequences_train)
X_val = pad_sequences(sequences_valid,maxlen=X_train.shape[1])
Y_train = to_categorical(y_train, 3)
Y_val = to_categorical(y_test, 3)
print('Shape of X train and X validation tensor:', X_train.shape,X_val.shape)
print('Shape of label train and validation tensor:', Y_train.shape,Y_val.shape)

Shape of X train and X validation tensor: (8, 30) (3, 30)
Shape of label train and validation tensor: (8, 3) (3, 3)


In [11]:
emblay = eu.word2vec_embedding_layer(pretrained_path, word_index=word_index)
#emblay = eu.glove_embedding_layer(pre_path, word_index=word_index, EMBEDDING_DIM=50)
sequence_length = X_train.shape[1]
model = kim_cnn(sequence_length, 300, embedding_layer=emblay, n_class=3)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 30, 300)      17400       input_2[0][0]                    
__________________________________________________________________________________________________
reshape_3 (Reshape)             (None, 30, 300, 1)   0           embedding_4[0][0]                
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 28, 1, 100)   90100       reshape_3[0][0]                  
__________________________________________________________________________________________________
conv2d_5 (

In [12]:
callbacks = [EarlyStopping(monitor='val_loss')]
model.fit(X_train, Y_train, batch_size=1000, epochs=10, verbose=1, validation_data=(X_val, Y_val),
         callbacks=callbacks)

Train on 8 samples, validate on 3 samples
Epoch 1/10
8/8 [==============================] - 1s 92ms/step - loss: 1.2122 - acc: 0.2500 - val_loss: 1.1602 - val_acc: 0.6667
Epoch 2/10
8/8 [==============================] - 0s 3ms/step - loss: 1.1257 - acc: 0.8750 - val_loss: 1.1036 - val_acc: 0.6667
Epoch 3/10
8/8 [==============================] - 0s 4ms/step - loss: 1.0396 - acc: 1.0000 - val_loss: 1.0504 - val_acc: 0.6667
Epoch 4/10
8/8 [==============================] - 0s 4ms/step - loss: 0.9557 - acc: 1.0000 - val_loss: 0.9973 - val_acc: 0.6667
Epoch 5/10
8/8 [==============================] - 0s 4ms/step - loss: 0.8881 - acc: 1.0000 - val_loss: 0.9438 - val_acc: 0.6667
Epoch 6/10
8/8 [==============================] - 0s 4ms/step - loss: 0.8364 - acc: 1.0000 - val_loss: 0.8914 - val_acc: 0.6667
Epoch 7/10
8/8 [==============================] - 0s 4ms/step - loss: 0.7344 - acc: 1.0000 - val_loss: 0.8425 - val_acc: 1.0000
Epoch 8/10
8/8 [==============================] - 0s 4ms/step